In [1]:
# ch4-2.py
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)

In [2]:
# 데이터로드 (NHIS_OPEN_GJ_2017.csv : 데이터 원본 파일)
# encoding : 윈도우즈 환경에서의 한글 처리
# engine : python 3.6에서 한글이 포함된 파일이름 사용
rawData_org = pd.read_csv('datasets/국민건강정보/NHIS_OPEN_GJ_2017.csv', encoding='CP949', engine='python')

In [3]:
# 원본데이터의 튜플 수와 컬럼수 출력
rawData_org.shape

(1000000, 34)

In [4]:
# 의사결정트리에 사용할 속성리스트
feature_columns_to_use = ['성별코드', '연령대코드(5세단위)', '신장(5Cm단위)', '체중(5Kg 단위)', '허리둘레', '시력(좌)', '시력(우)',
                    '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '트리글리세라이드', 'HDL콜레스테롤',
                    'LDL콜레스테롤', '요단백', '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT', '감마지티피', '흡연상태', '음주여부']

rawData = rawData_org[feature_columns_to_use]

In [5]:
# 속성값이 NULL인 튜플 제외
rawData = rawData.dropna()

# 인덱스 재설정
rawData.reset_index(inplace=True, drop=True)

# 원본데이터의 튜플 수와 컬럼수 출력
rawData.shape

(990910, 22)

In [6]:
import random as rd  # 샘플링을 위한 random 패키지 임포트

# 1) 샘플링을 통한 수량 축소
# 원본데이터 중 50,000개를 튜플만 샘플링
# len() : 데이터프레임의 크기를 구하는 함수
sample_idx = rd.sample(range(0, len(rawData)), 50000)

# 인덱스 값 정렬(오름차순)
sample_idx.sort()

sample_idx

[56,
 58,
 69,
 125,
 136,
 179,
 188,
 207,
 243,
 319,
 339,
 407,
 439,
 455,
 466,
 468,
 473,
 493,
 503,
 504,
 547,
 554,
 564,
 565,
 593,
 634,
 635,
 678,
 679,
 696,
 701,
 816,
 834,
 839,
 868,
 903,
 916,
 932,
 936,
 939,
 951,
 985,
 1005,
 1007,
 1010,
 1023,
 1032,
 1033,
 1063,
 1070,
 1087,
 1089,
 1171,
 1184,
 1251,
 1412,
 1442,
 1450,
 1459,
 1494,
 1513,
 1517,
 1543,
 1582,
 1583,
 1590,
 1596,
 1600,
 1625,
 1629,
 1638,
 1690,
 1725,
 1731,
 1809,
 1837,
 1852,
 1882,
 1891,
 1986,
 1989,
 1993,
 2041,
 2050,
 2064,
 2066,
 2069,
 2072,
 2093,
 2112,
 2159,
 2213,
 2271,
 2272,
 2300,
 2346,
 2352,
 2365,
 2377,
 2382,
 2447,
 2554,
 2598,
 2646,
 2647,
 2650,
 2668,
 2703,
 2723,
 2750,
 2772,
 2777,
 2778,
 2785,
 2797,
 2799,
 2829,
 2842,
 2869,
 2872,
 2899,
 2905,
 2916,
 2929,
 2932,
 2965,
 3009,
 3058,
 3071,
 3100,
 3110,
 3115,
 3117,
 3120,
 3149,
 3180,
 3187,
 3194,
 3227,
 3240,
 3289,
 3304,
 3347,
 3365,
 3366,
 3466,
 3470,
 3528,
 3547,
 3

In [7]:
# 샘플링 된 인덱스로 구성된 샘플 데이터프레임 생성
rawData_sample = rawData.loc[sample_idx]

rawData_sample.reset_index(inplace=True, drop=True) #  인덱스 재설정

# 첫 10개의 행만 출력
rawData_sample.head(10)

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg 단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,음주여부
0,2,6,155.0,65.0,74.0,0.3,0.3,1.0,1.0,99.0,...,56.0,61.0,57.0,1.0,0.6,19.0,5.0,10.0,1.0,1.0
1,1,9,175.0,80.0,84.0,1.2,1.0,1.0,1.0,130.0,...,97.0,52.0,127.0,1.0,1.1,29.0,22.0,25.0,2.0,1.0
2,2,12,145.0,45.0,70.0,1.0,0.7,1.0,1.0,112.0,...,68.0,70.0,120.0,1.0,0.7,25.0,23.0,11.0,1.0,1.0
3,2,6,160.0,55.0,70.0,1.2,0.9,1.0,1.0,110.0,...,52.0,54.0,125.0,1.0,0.8,17.0,13.0,14.0,1.0,1.0
4,2,13,160.0,55.0,78.0,0.8,0.7,2.0,1.0,130.0,...,104.0,64.0,105.0,1.0,0.7,27.0,20.0,16.0,3.0,1.0
5,2,9,160.0,75.0,96.0,0.7,1.0,1.0,1.0,176.0,...,44.0,69.0,116.0,1.0,0.9,52.0,69.0,29.0,2.0,1.0
6,1,10,165.0,60.0,84.5,1.0,1.0,1.0,1.0,120.0,...,402.0,51.0,142.0,1.0,1.0,34.0,87.0,65.0,3.0,0.0
7,2,10,145.0,45.0,79.0,1.2,1.2,1.0,1.0,120.0,...,289.0,61.0,124.0,1.0,0.8,23.0,15.0,14.0,1.0,1.0
8,1,15,160.0,65.0,82.0,0.7,0.7,1.0,1.0,125.0,...,145.0,49.0,105.0,1.0,1.0,18.0,18.0,22.0,1.0,0.0
9,2,15,150.0,45.0,82.0,0.6,0.5,2.0,1.0,132.0,...,307.0,32.0,1.0,1.0,0.8,27.0,21.0,27.0,1.0,0.0


In [8]:
rawData_sample.shape

(50000, 22)

In [9]:
# nonnumeric 속성을 categrical 데이터로 변환
nonnumeric_columns = ['성별코드','음주여부']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for feature in nonnumeric_columns:
    rawData_sample[feature] = le.fit_transform(rawData_sample[feature])   

In [10]:
# 소스 데이터프레임에서 분류(classification)을 위한 속성 집합
X = rawData_sample.loc[:, feature_columns_to_use[:-1]]  
y = rawData_sample.loc[:, '음주여부']  # 분류 클래스(class)

from sklearn.model_selection import train_test_split  # 분석모형 선택에 관련된 모듈
    
# 자동으로 데이터셋을 트레이닝셋과 테스트셋으로 분리해주는 함수로
# 트레이닝셋과 데이터셋의 비율을 7:3으로 세팅함
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [11]:
# RMSLE 계산하는 사용자정의 함수
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values) :
    # 넘파이로 배열 형태로 변환
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 예측값과 실제값에 1을 더하고 로그를 씌움
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)
    
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 함
    difference = log_predict - log_actual
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)
    
    # 평균값 구함
    mean_difference = difference.mean()
    
    # 다시 루트를 씌움
    score = np.sqrt(mean_difference)
    
    return score

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [12]:
# Scikit-Learn 패키지 : 머신 러닝 교육 및 실무를 위한 패키지로 샘플 데이터셋,
# 다양한 기계학습 기법에 대한 함수 등을 포함하고 있음
from sklearn.ensemble import RandomForestClassifier  # 랜덤포리스트 기법에 관련된 모듈

# DecisionTreeClassifier() : 의사결정트리를 생성하는 함수
random_forest = RandomForestClassifier(n_estimators=20, random_state=0)
random_forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [13]:
# KFold 교차검증
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

# 트레이닝셋에 대한 KFold 교차검증 수
%time score = cross_val_score(random_forest, X_train, y_train, cv=k_fold, scoring=rmsle_scorer)
score = score.mean()

# 0에 근접할수록 좋은 데이터
print("Score= {0:.5f}".format(score))

Wall time: 25.7 s
Score= 0.36971


In [17]:
# fit() : 트레이닝 데이터셋을 대상으로 의사결정트리 학습 진행
random_forest.fit(X_train, y_train)

# tree.predict() 함수를 활용하여 의사결정트리를 대상으로 테스트셋을 예측
y_pred_tr = random_forest.predict(X_test)

# predict_proba() : 각 input 개체에 대한 예측클래스 확률 구함
y_pred_tr_prob = random_forest.predict_proba(X_test)

In [18]:
# 점수 출력
print("Train Set Score1 : {:.2f}".format(random_forest.score(X_train, y_train)))
print("Test Set Score1 : {:.2f}".format(random_forest.score(X_test, y_test)))

Train Set Score1 : 1.00
Test Set Score1 : 0.71


In [20]:
from sklearn.metrics import accuracy_score  # 분류 정확도(classification accuracy)를 계산하는 모듈

# accuracy_score() 함수를 활용하여 테스트셋의 실제 클래스와 예측된 클래스 간 정확도 측정
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred_tr))

Accuracy: 0.71


In [22]:
# 속성(feature) 별 중요도를 저장하는 데이터프레임 생성 
sel_feature = pd.DataFrame({'중요도' : random_forest.feature_importances_}, index = feature_columns_to_use[:-1])

# 중요도의 내림차순으로 정렬
sel_feature.sort_values(by='중요도', ascending=False)

,중요도
감마지티피,0.091488
연령대코드(5세단위),0.078381
HDL콜레스테롤,0.069071
트리글리세라이드,0.065364
LDL콜레스테롤,0.062608
신장(5Cm단위),0.059850
식전혈당(공복혈당),0.056322
(혈청지오티)ALT,0.055432
흡연상태,0.054905
허리둘레,0.053207


In [23]:
# 예측결과와 실제결과 비교
appData_anal_comp = pd.DataFrame(data={'actual': y_test, 'pred': y_pred_tr}, columns={'actual', 'pred'})
appData_anal_comp

,actual,pred
11841,1,0
19602,1,0
45519,1,1
25747,0,0
42642,0,1
...,...,...
38344,1,1
49984,1,1
32624,0,0
46437,1,1


In [24]:
# 예측 클래스와 클래스 예측확률을 X_test 데이터프레임에 붙임
X_test['음주여부'] = y_test
X_test['음주여부예측'] = y_pred_tr

X_test['음주확률'] = y_pred_tr_prob[:,1]

X_test

C:\Users\a\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\a\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\a\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg 단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,LDL콜레스테롤,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,음주여부,음주여부예측,음주확률
11841,0,12,155.0,60.0,88.5,0.2,0.6,1.0,1.0,110.0,...,50.0,1.0,0.8,23.0,20.0,19.0,3.0,1,0,0.50
19602,1,8,155.0,50.0,62.0,1.2,1.0,1.0,1.0,118.0,...,86.0,1.0,0.7,15.0,7.0,11.0,1.0,1,0,0.40
45519,0,16,160.0,55.0,76.0,1.0,0.1,1.0,1.0,125.0,...,104.0,1.0,1.2,47.0,27.0,115.0,1.0,1,1,0.55
25747,0,18,155.0,55.0,78.0,0.3,0.3,2.0,1.0,138.0,...,117.0,1.0,1.1,28.0,23.0,13.0,2.0,0,0,0.30
42642,1,11,165.0,60.0,75.0,1.5,1.5,1.0,1.0,162.0,...,84.0,1.0,0.9,54.0,106.0,68.0,1.0,0,1,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38344,1,6,160.0,55.0,70.0,1.2,1.5,1.0,1.0,119.0,...,105.0,1.0,0.7,10.0,9.0,10.0,1.0,1,1,0.75
49984,1,8,160.0,45.0,62.0,1.2,0.9,1.0,1.0,102.0,...,88.0,1.0,0.7,21.0,13.0,26.0,1.0,1,1,0.80
32624,0,11,160.0,55.0,74.0,0.8,0.8,1.0,1.0,109.0,...,112.0,1.0,0.9,18.0,14.0,15.0,3.0,0,0,0.30
46437,1,6,155.0,55.0,68.0,1.0,1.0,1.0,1.0,112.0,...,53.0,1.0,0.6,19.0,20.0,13.0,1.0,1,1,0.60
